In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.a2c import A2C
from networks.a2c.actor import Actor
from networks.a2c.critic import Critic

import matplotlib.pyplot as plt
import torch

norm_cols = ['Volume XRP']
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 15
n_features = 3

env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-02-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

actor_model = Actor(
    state_dim=batch_dur*n_features+3,
    action_dim=3,
)
critic_model = Critic(state_dim=batch_dur*n_features+3)
a2c = A2C(
    env=env, 
    actor=actor_model,
    critic=critic_model,
    n_actns=3,
    actor_optmz=torch.optim.Adam(actor_model.parameters(), lr=0.0008),
    critic_optmz=torch.optim.Adam(critic_model.parameters(), lr=0.0005),
    hyprprms={
        'gamma': 0.9995,
    },
    log_freq=1,
    mdl_pth='../models/a2c_nn_3',
    load_models=False,
)

In [ ]:
a2c.run(1000)


Ep: 0 | TS: 8428 | L: 1.111 | R: -1.65 | P: 36.89 | R.Avg P: 36.89 | NW: 142.79 | R.Avg NW: 142.79 | R.U: 335
Ep: 1 | TS: 14194 | L: 1.184 | R: -9.93 | P: -4.88 | R.Avg P: 16.0 | NW: 96.88 | R.Avg NW: 119.84 | R.U: 280
Ep: 2 | TS: 59346 | L: 0.923 | R: -7.91 | P: 11.01 | R.Avg P: 14.34 | NW: 115.02 | R.Avg NW: 118.23 | R.U: 309
Ep: 3 | TS: 95400 | L: 1.087 | R: -10.14 | P: -7.46 | R.Avg P: 8.89 | NW: 94.58 | R.Avg NW: 112.32 | R.U: 306
Ep: 4 | TS: 108982 | L: 1.245 | R: -8.61 | P: 33.21 | R.Avg P: 13.75 | NW: 134.62 | R.Avg NW: 116.78 | R.U: 320
Ep: 5 | TS: 131795 | L: 1.261 | R: -9.74 | P: 8.96 | R.Avg P: 12.95 | NW: 110.19 | R.Avg NW: 115.68 | R.U: 322
Ep: 6 | TS: 148850 | L: 1.242 | R: -9.23 | P: 22.68 | R.Avg P: 14.34 | NW: 123.57 | R.Avg NW: 116.81 | R.U: 328
Ep: 7 | TS: 170822 | L: 1.132 | R: -9.59 | P: 11.76 | R.Avg P: 14.02 | NW: 112.68 | R.Avg NW: 116.29 | R.U: 330
Ep: 8 | TS: 214563 | L: 1.039 | R: -8.73 | P: 29.81 | R.Avg P: 15.78 | NW: 130.75 | R.Avg NW: 117.9 | R.U: 340
E

In [ ]:
rewards, profits, actions = a2c.evaluate(start_dt='2021-01-30 15:15', duration=500, show_pred=True)

In [ ]:
rewards, profits, actions = a2c.evaluate(start_dt='2021-01-31 15:15', duration=500, show_pred=True)

## Visualizations

### Training 

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

In [ ]:
# torch.save(a2c.actor.state_dict(), '../models/a2c_nn_3/actor')
# torch.save(a2c.critic.state_dict(), '../models/a2c_nn_3/critic')

In [ ]:
# import pickle
# with open("../pickles/a2c_logs_main.pickle","wb") as f:
#     temp = dict(a2c.logs)
#     pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)